In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers -qq
!pip install sentencepiece -qq
!pip install tokenizer -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.8 MB/s eta 0:00:00


##Load Dataset

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('hsd_merge_cleaned_lowered.csv')

#data.rename(columns={"Message Text": "text", "Toxic/Not Toxic": "label"}, inplace=True)

train = data[: int(len(data) * 0.7)]
valid = data[int(len(data) * 0.7): int(len(data) * 0.8)]
test = data[int(len(data) * 0.8): int(len(data) * 0.1)]

X_train = train['text']
y_train = train['label']

X_valid = valid['text']
y_valid = valid['label']

X_test = test['text']
y_test = test['label']

In [26]:
from sklearn.model_selection import train_test_split

def to_df(x, y):
    d = {"text": x, "label": y}
    return pd.DataFrame(d)

def split_3(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = df["text"].copy()
    y = df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y)
    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

import numpy as np
import pandas as pd

#tname_data = "./hsd_merged_cleaned_lowered"
#data = pd.read_csv(f"{tname_data}.csv")
data = pd.read_csv('hsd_merge_cleaned_lowered.csv')

train, valid, test = split_3(data)

X_train = train['text']
y_train = train['label']

X_valid = valid['text']
y_valid = valid['label']

X_test = test['text']
y_test = test['label']

In [27]:

# import numpy as np
# import pandas as pd

# train = pd.read_csv('/content/train.csv')
# dev = pd.read_csv('/content/dev.csv')
# test = pd.read_csv('/content/test.csv')


# train['free_text'] = train['free_text'].astype(str)
# X_train = train['free_text']
# y_train = train["label_id"]

# dev['free_text'] = dev['free_text'].astype(str)
# X_valid = dev['free_text']
# y_valid = dev['label_id']

# test['free_text'] = test['free_text'].astype(str)
# X_test = test['free_text']
# y_test = test['label_id']


## Set Cuda

In [28]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.cuda.is_available()

True

# Extract feature by using BETO

In [29]:
import pandas as pd
from glob import glob

train_sentences = list(train['text'].values)
train_labels = list(train['label'].values)

valid_sentences = list(valid['text'].values)
valid_labels = list(valid['label'].values)

test_sentences = list(test['text'].values)
test_labels = list(test['label'].values)

Load tokenizer of BETO

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

In [31]:
#choose max_length for phobert model based on the input length

max_length = 0
list_len=[]
for sentence in train_sentences:
    length = len(tokenizer.tokenize(sentence))
    list_len.append(length)
    
from collections import Counter
Counter(list_len).most_common(100)

[(16, 359),
 (21, 354),
 (14, 343),
 (15, 340),
 (12, 334),
 (18, 333),
 (23, 328),
 (11, 328),
 (19, 324),
 (13, 321),
 (20, 320),
 (22, 315),
 (17, 293),
 (10, 292),
 (9, 292),
 (24, 289),
 (26, 276),
 (25, 271),
 (8, 240),
 (27, 236),
 (28, 231),
 (29, 212),
 (30, 206),
 (7, 196),
 (6, 169),
 (31, 168),
 (32, 151),
 (33, 138),
 (5, 114),
 (34, 113),
 (35, 107),
 (38, 99),
 (43, 94),
 (53, 91),
 (48, 90),
 (4, 89),
 (37, 89),
 (49, 88),
 (40, 87),
 (42, 87),
 (41, 85),
 (50, 84),
 (44, 83),
 (45, 81),
 (39, 80),
 (51, 78),
 (46, 78),
 (47, 77),
 (36, 76),
 (54, 73),
 (52, 70),
 (55, 63),
 (56, 61),
 (57, 47),
 (58, 45),
 (59, 41),
 (3, 40),
 (61, 26),
 (60, 24),
 (62, 20),
 (63, 16),
 (64, 15),
 (65, 14),
 (2, 12),
 (66, 7),
 (1, 7),
 (67, 6),
 (72, 3),
 (68, 3),
 (70, 3),
 (79, 2),
 (69, 2),
 (74, 1),
 (84, 1),
 (71, 1),
 (85, 1)]

In [32]:
# Encode train label

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels)
encoded_labels = le.transform(train_labels)
encoded_test_labels = le.transform(valid_labels)

In [33]:
# Tokens IDs tensor

def encoder_generator(sentences,labels):
    
    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):
        
        sent_index.append(index)
        
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=20,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0).cuda()
    attention_masks = torch.cat(attention_masks,dim=0).cuda()
    labels = torch.tensor(labels).cuda()
    sent_index = torch.tensor(sent_index).cuda()

    return sent_index,input_ids,attention_masks,labels

train_sent_index,train_input_ids,train_attention_masks,train_encoded_label_tensors = encoder_generator(train_sentences,encoded_labels)
valid_sent_index,valid_input_ids,valid_attention_masks,valid_encoded_label_tensors = encoder_generator(valid_sentences,encoded_test_labels)
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

C:\ProgramData\Anaconda3\envs\anti-disc\lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  chilenos que apoyan keiko saco weas qls
Token IDs: tensor([    4, 25487,  1041, 17341,  4926, 19895, 12005,  1005,  2756,  5343,
         8856,     5,     1,     1,     1,     1,     1,     1,     1,     1],
       device='cuda:0')


In [34]:
# Connvert train, dev input by using TensorDataset

from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(train_input_ids,train_attention_masks,train_encoded_label_tensors)
valid_dataset = TensorDataset(valid_input_ids,valid_attention_masks,valid_encoded_label_tensors)

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(valid_dataset)))

train data samples is 10133
valid data samples is 2534


In [35]:
# Set cuda by using device

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(valid_dataset,
                              sampler=RandomSampler(valid_dataset),
                              batch_size=bs)

Load model BETO

In [36]:
from transformers import AutoModel

beto = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
beto = beto.to(device)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Build CNN 

In [37]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class CNNForNLP(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, num_filters, filter_sizes):
        super(CNNForNLP, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv1d(embedding_dim, num_filters, filter_size)
            for filter_size in filter_sizes
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(len(filter_sizes) * num_filters, num_classes)
    
    def forward(self, x, _):
        embedded = self.embedding(x)  # x: (batch_size, sequence_length)
        embedded = embedded.permute(0, 2, 1)  # embedded: (batch_size, embedding_dim, sequence_length)
        feature_maps = []
        for conv in self.convs:
            feature_map = torch.relu(conv(embedded))  # feature_map: (batch_size, num_filters, H)
            pooled = torch.max(feature_map, dim=2)[0]  # pooled: (batch_size, num_filters)
            feature_maps.append(pooled)
        combined = torch.cat(feature_maps, dim=1)  # combined: (batch_size, len(filter_sizes) * num_filters)
        combined = self.dropout(combined)
        logits = self.fc(combined)  # logits: (batch_size, num_classes)
        return logits


In [38]:

# Definir los parámetros del modelo
vocab_size = 31002#tamaño del vocabulario
embedding_dim = 768 #Dimension de los vectores de embedding 
num_classes = 2 #numero de clases o categorias de clasificacion
num_filters = 32  #numero de filtros convolucionales
filter_sizes = [3]  #tamaño de los filtros convolucionales


CNNmodel = CNNForNLP(vocab_size,embedding_dim,num_classes,num_filters,filter_sizes)


In [39]:

# Imprimir el modelo
print(CNNmodel)

CNNForNLP(
  (embedding): Embedding(31002, 768)
  (convs): ModuleList(
    (0): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)


In [40]:
# Optimizer and criterion

import torch.optim as optim

model_parameters = list(CNNmodel.parameters())

optimizer = optim.Adam(model_parameters,lr=2e-5,eps=1e-8)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [41]:
# Calculate accuracy per batch during train

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()

In [42]:
# Def for training

from tqdm import tqdm

def train(model):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        optimizer.zero_grad()
        
        predictions = model(b_input_ids,b_input_mask)
        
        loss = criterion(predictions, b_labels)
        
        acc = categorical_accuracy(predictions, b_labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(train_data_loader), epoch_acc / len(train_data_loader)

In [43]:
# Class for predict label

import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [44]:
# Evaluate loss, acc  and f1-macro

from sklearn.metrics import classification_report,accuracy_score,f1_score
def eval(model):
    epoch_loss = 0
    
    total_predictions = []
    total_true = []
    
    all_true_labels = []
    all_pred_labels = []
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in tqdm(valid_data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            predictions = model(b_input_ids,b_input_mask)

            loss = criterion(predictions, b_labels)
            epoch_loss += loss.item()
            
            predictions = predictions.detach().cpu().numpy()

            label_ids = b_labels.to('cpu').numpy()
    
            pred,true = predictions_labels(predictions,label_ids)
        
            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')
    
    avg_val_loss = epoch_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
            
    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [45]:
# Time for training

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [46]:
# Set device and gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

CNNmodel.cuda()

CNNForNLP(
  (embedding): Embedding(31002, 768)
  (convs): ModuleList(
    (0): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

# Training

In [47]:
epochs = 20

best_macro_f1 = float('0')

for epoch in range(epochs):
    
    start_time = time.time()
    train_loss,train_acc = train(CNNmodel)
    valid_loss,valid_acc,macro_f1 = eval(CNNmodel)
    end_time = time.time()
    
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
    torch.save(CNNmodel,'./cnn_model_part1_'+'task2a_2.pt')
    print("model saved")
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============')

100%|██████████| 20/20 [00:00<00:00, 588.27it/s]


              precision    recall  f1-score   support

           0       1.00      0.86      0.92      2526
           1       0.01      0.25      0.01         8

    accuracy                           0.86      2534
   macro avg       0.50      0.56      0.47      2534
weighted avg       0.99      0.86      0.92      2534

accuracy = 0.86
model saved
Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 0.652 | Train acc: 63.21%
	 Val. Loss: 0.461 |  Val. acc: 85.95%
	 Val. Loss: 0.461 |  Val. F1: 46.77%
=============Epoch Ended==============


100%|██████████| 20/20 [00:00<00:00, 625.59it/s]
C:\ProgramData\Anaconda3\envs\anti-disc\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\anti-disc\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\anti-disc\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

              precision    recall  f1-score   support

           0       1.00      0.86      0.93      2534
           1       0.00      0.00      0.00         0

    accuracy                           0.86      2534
   macro avg       0.50      0.43      0.46      2534
weighted avg       1.00      0.86      0.93      2534

accuracy = 0.86
model saved
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.501 | Train acc: 78.38%
	 Val. Loss: 0.410 |  Val. acc: 86.11%
	 Val. Loss: 0.410 |  Val. F1: 46.27%
=============Epoch Ended==============


100%|██████████| 20/20 [00:00<00:00, 588.32it/s]
C:\ProgramData\Anaconda3\envs\anti-disc\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\anti-disc\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\envs\anti-disc\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

...

              precision    recall  f1-score   support

           0       1.00      0.87      0.93      2506
           1       0.05      0.68      0.10        28

    accuracy                           0.87      2534
   macro avg       0.52      0.77      0.51      2534
weighted avg       0.99      0.87      0.92      2534

accuracy = 0.87
model saved
Epoch: 18 | Epoch Time: 0m 0s
	Train Loss: 0.375 | Train acc: 85.94%
	 Val. Loss: 0.365 |  Val. acc: 86.50%
	 Val. Loss: 0.365 |  Val. F1: 51.35%
=============Epoch Ended==============


100%|██████████| 20/20 [00:00<00:00, 624.98it/s]


              precision    recall  f1-score   support

           0       0.99      0.87      0.93      2502
           1       0.06      0.62      0.10        32

    accuracy                           0.86      2534
   macro avg       0.53      0.75      0.52      2534
weighted avg       0.98      0.86      0.92      2534

accuracy = 0.86
model saved
Epoch: 19 | Epoch Time: 0m 0s
	Train Loss: 0.367 | Train acc: 86.17%
	 Val. Loss: 0.363 |  Val. acc: 86.42%
	 Val. Loss: 0.363 |  Val. F1: 51.54%
=============Epoch Ended==============


100%|██████████| 20/20 [00:00<00:00, 588.24it/s]


              precision    recall  f1-score   support

           0       0.99      0.87      0.93      2502
           1       0.06      0.66      0.11        32

    accuracy                           0.87      2534
   macro avg       0.53      0.76      0.52      2534
weighted avg       0.98      0.87      0.92      2534

accuracy = 0.87
model saved
Epoch: 20 | Epoch Time: 0m 0s
	Train Loss: 0.366 | Train acc: 86.10%
	 Val. Loss: 0.361 |  Val. acc: 86.50%
	 Val. Loss: 0.361 |  Val. F1: 51.82%
=============Epoch Ended==============


In [48]:
# Save BETO and CNN 

torch.save(CNNmodel,'module2_part1.pt')


## EVALUATING

In [50]:
# Load phobert and cnn

import torch
CNNmodel = torch.load(r'cnn_model_part1_task2a_2.pt')
CNNmodel.eval()



CNNForNLP(
  (embedding): Embedding(31002, 768)
  (convs): ModuleList(
    (0): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

Predict label from true label

In [51]:
test_sent_index, test_input_ids, test_attention_masks, test_encoded_label_tensors = encoder_generator(test_sentences,test_labels)
test_dataset = TensorDataset(test_input_ids,test_attention_masks,test_encoded_label_tensors)

test_data_loader = DataLoader(test_dataset,
                              sampler=RandomSampler(test_dataset),
                              batch_size=bs)

all_pred_labels = []
all_true_labels = []

with torch.no_grad():
  for batch in tqdm(test_data_loader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    predictions = CNNmodel(b_input_ids,b_input_mask)

    
    predictions = predictions.detach().cpu().numpy()

    label_ids = b_labels.to('cpu').numpy()

    pred, true = predictions_labels(predictions, label_ids)

    all_pred_labels.extend(pred)
    all_true_labels.extend(true)

C:\ProgramData\Anaconda3\envs\anti-disc\lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 25/25 [00:00<00:00, 79.12it/s]


In [52]:
# The final score in the test set (classification report)

print(classification_report(all_pred_labels,all_true_labels, digits = 4))

              precision    recall  f1-score   support

           0     0.9971    0.8729    0.9308      3115
           1     0.1000    0.8462    0.1789        52

    accuracy                         0.8724      3167
   macro avg     0.5485    0.8595    0.5549      3167
weighted avg     0.9823    0.8724    0.9185      3167



In [53]:
# Confusion matrix in thetest set

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_true_labels, all_pred_labels)
cm

array([[2719,    8],
       [ 396,   44]], dtype=int64)